In [11]:
import pandas as pd
import numpy as np
import openpyexcel
import pickle

In [3]:
df = pd.read_csv("layoffs.csv")
df["percentage_laid_off"].fillna(0,inplace=True)
df["funds_raised"].fillna(0,inplace=True)
df["date"] = df["date"].convert_dtypes(convert_string=True)
df['year'] = df['date'].apply(lambda x: int(x.split('-')[0]))

years = df["year"].unique()
dfs=[]
for year in years:
    dfs.append(df[df["year"]==year])
df



,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised,year
0,New Work,Hamburg,Consumer,400.0,0.00,2024-01-11,Post-IPO,Germany,0.0,2024
1,Playtika,Tel Aviv,Consumer,300.0,0.10,2024-01-11,Post-IPO,Israel,0.0,2024
2,Discord,SF Bay Area,Consumer,170.0,0.17,2024-01-11,Series H,United States,995.0,2024
3,Inmobi,Bengaluru,Marketing,125.0,0.05,2024-01-11,Unknown,India,320.0,2024
4,Audible,New York City,Media,100.0,0.05,2024-01-11,Acquired,United States,14.0,2024
...,...,...,...,...,...,...,...,...,...,...
3308,Service,Los Angeles,Travel,NaN,1.00,2020-03-16,Seed,United States,5.1,2020
3309,HopSkipDrive,Los Angeles,Transportation,8.0,0.10,2020-03-13,Unknown,United States,45.0,2020
3310,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0,2020
3311,Tamara Mellon,Los Angeles,Retail,20.0,0.40,2020-03-12,Series C,United States,90.0,2020


In [4]:
layoff_c_dict ={}
layoff_i_dict ={}
funds_c_dict ={}
funds_i_dict ={}

for i in range(1,len(years)):
    
    sum_per_company = dfs[i].groupby(['company', 'industry'])['percentage_laid_off'].agg(["sum", "std"]).reset_index()
    sum_per_company["std"].fillna(0,inplace=True)
    avg_per_category = sum_per_company.groupby(['industry'])['sum'].agg(['mean', 'std']).reset_index()
    avg_per_category["std"].fillna(0,inplace=True)
    
    layoff_c_dict[years[i]] = {
       
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(sum_per_company['company'], sum_per_company['sum'], sum_per_company['std'])
        }
    
    layoff_i_dict[years[i]]={
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(avg_per_category['industry'], avg_per_category['mean'], avg_per_category['std'])
        }
    
  

    funds_per_company = dfs[i].groupby(['company', 'industry'])['funds_raised'].agg(["sum", "std"]).reset_index()
    funds_per_company["std"].fillna(0,inplace=True)
    avgf_per_category = funds_per_company.groupby(['industry'])['sum'].agg(['mean', 'std']).reset_index()
    avgf_per_category["std"].fillna(0,inplace=True)

    funds_c_dict[years[i]] = {
       
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(funds_per_company['company'], funds_per_company['sum'], funds_per_company['std'])
        }
    
    funds_i_dict[years[i]]={
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(avgf_per_category['industry'], avgf_per_category['mean'], avgf_per_category['std'])
        }
    


In [5]:
df2 =pd.read_csv("layoffs_data(3).csv")
df2["Percentage"].fillna(0,inplace=True)
df2["Funds_Raised"].fillna(0,inplace=True)
df2["Date"] = df2["Date"].convert_dtypes(convert_string=True)
df2['year'] = df2['Date'].apply(lambda x: int(x.split('-')[0]))
years = df2["year"].unique()
df2s=[]
for year in years:
    df2s.append(df2[df2["year"]==year])
df2


,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,year
0,Fisker,Los Angeles,Transportation,0.15,2024-02-29,https://techcrunch.com/2024/02/29/fisker-layof...,1700.0,Post-IPO,2024-02-29 23:35:20,United States,NaN,Unknown,2024
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-com...,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown,2024
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/po...,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown,2024
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prim...,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown,2024
4,Motional,Boston,Transportation,0.05,2024-02-28,https://techcrunch.com/2024/02/28/hyundai-back...,0.0,Unknown,2024-02-28 18:50:58,United States,NaN,Unknown,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,Service,Los Angeles,Travel,1.00,2020-03-16,https://techcrunch.com/2020/03/16/travel-savin...,5.1,Seed,2020-08-31 15:17:26,United States,NaN,Unknown,2020
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-la...,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown,2020
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/123854...,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown,2020
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown,2020


In [6]:
layoff_c_dict2 ={}
layoff_i_dict2 ={}
funds_c_dict2 ={}
funds_i_dict2 ={}

for i in range(1,len(years)):
    
    sum_per_company = df2s[i].groupby(['Company', 'Industry'])['Percentage'].agg(["sum", "std"]).reset_index()
    sum_per_company["std"].fillna(0,inplace=True)
    avg_per_category = sum_per_company.groupby(['Industry'])['sum'].agg(['mean', 'std']).reset_index()
    avg_per_category["std"].fillna(0,inplace=True)
    
    layoff_c_dict2[years[i]] = {
       
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(sum_per_company['Company'], sum_per_company['sum'], sum_per_company['std'])
        }
    
    layoff_i_dict2[years[i]]={
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(avg_per_category['Industry'], avg_per_category['mean'], avg_per_category['std'])
        }
    
    
    funds_per_company = df2s[i].groupby(['Company', 'Industry'])['Funds_Raised'].agg(["sum", "std"]).reset_index()
    funds_per_company["std"].fillna(0,inplace=True)
    avgf_per_category = funds_per_company.groupby(['Industry'])['sum'].agg(['mean', 'std']).reset_index()
    avgf_per_category["std"].fillna(0,inplace=True)
    funds_c_dict2[years[i]] = {
       
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(funds_per_company['Company'], funds_per_company['sum'], funds_per_company['std'])
        }
    
    funds_i_dict2[years[i]]={
            company: (sum_value, std_value)
            for company, sum_value, std_value in zip(avgf_per_category['Industry'], avgf_per_category['mean'], avgf_per_category['std'])
        }


In [7]:
#demp
print(f"percentage of layoffs at F-secure in 2023 : {layoff_c_dict2[2023][" F-Secure "][0]}, standard diviation of the same compnay same year: {layoff_c_dict2[2023][" F-Secure "][1]}")
print(f"percentage of layoffs inth AI sector in 2023 : {layoff_i_dict2[2023]["AI"][0]}, standard diviation in the same Industry same year: {layoff_i_dict2[2023]["AI"][1]}")

percentage of layoffs at F-secure in 2023 : 0.0, standard diviation of the same compnay same year: 0.0
percentage of layoffs inth AI sector in 2023 : 0.05714285714285715, standard diviation in the same Industry same year: 0.09759000729485333


In [9]:
df = pd.read_csv("output/avg_percentage_per_category_tabel_2_2023.csv")
df["Industry"].to_csv("industry.csv")

In [3]:
df = pd.read_csv("layoffs.csv")
df

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,New Work,Hamburg,Consumer,400.0,NaN,2024-01-11,Post-IPO,Germany,NaN
1,Playtika,Tel Aviv,Consumer,300.0,0.10,2024-01-11,Post-IPO,Israel,NaN
2,Discord,SF Bay Area,Consumer,170.0,0.17,2024-01-11,Series H,United States,995.0
3,Inmobi,Bengaluru,Marketing,125.0,0.05,2024-01-11,Unknown,India,320.0
4,Audible,New York City,Media,100.0,0.05,2024-01-11,Acquired,United States,14.0
...,...,...,...,...,...,...,...,...,...
3308,Service,Los Angeles,Travel,NaN,1.00,2020-03-16,Seed,United States,5.1
3309,HopSkipDrive,Los Angeles,Transportation,8.0,0.10,2020-03-13,Unknown,United States,45.0
3310,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0
3311,Tamara Mellon,Los Angeles,Retail,20.0,0.40,2020-03-12,Series C,United States,90.0


In [4]:
df = pd.read_csv("layoffs_data(3).csv")
df

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
0,Fisker,Los Angeles,Transportation,0.15,2024-02-29,https://techcrunch.com/2024/02/29/fisker-layof...,1700.0,Post-IPO,2024-02-29 23:35:20,United States,NaN,Unknown
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-com...,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/po...,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prim...,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown
4,Motional,Boston,Transportation,0.05,2024-02-28,https://techcrunch.com/2024/02/28/hyundai-back...,NaN,Unknown,2024-02-28 18:50:58,United States,NaN,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
3480,Service,Los Angeles,Travel,1.00,2020-03-16,https://techcrunch.com/2020/03/16/travel-savin...,5.1,Seed,2020-08-31 15:17:26,United States,NaN,Unknown
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-la...,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/123854...,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown


In [15]:
df = pd.read_excel("tech_layoffs.xlsx")
df

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200,2023-12-20,15.0,1333,1133,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100,2023-12-19,20.0,500,400,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350,2023-12-18,10.0,3500,3150,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100,2023-12-18,10.0,1000,900,Retail,Unknown,1500,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900,2023-12-14,24.0,3750,2850,Transportation,Acquired,$15000,2023,37.77493,-122.41942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,3286,Pitch,Berlin,Germany,Europe,80,2024-08-01,67.0,119,39,Other,Series B,$138,2024,52.52437,13.41053
1414,3287,BenchSci,Toronto,Canada,North America,70,2024-08-01,17.0,412,342,Healthcare,Series D,$164,2024,43.70643,-79.39864
1415,3288,NanoString Technology,Seattle,USA,North America,50,2024-08-01,9.0,556,506,Healthcare,Post-IPO,$826,2024,47.60621,-122.33207
1416,3289,Trigo,Tel Aviv,Israel,Asia,30,2024-04-01,15.0,200,170,Retail,Series C,$199,2024,32.08088,34.78057


In [17]:
df = pd.read_csv("layoffs_data.csv")
df

,Company,Location_HQ,Industry,Laid_Off_Count,Percentage,Date,Source,Funds_Raised (Millions),Stage,Date_Added,Country,List_of_Employees_Laid_Off,Sum of Employees,Average # of Employees Laid Off,Average Layoff %,Total Funds Raised,Average Funds Raised,Unnamed: 17,Unnamed: 18
0,Splunk,SF Bay Area,Data,325,4%,2/1/2023,https://www.marketwatch.com/story/splunk-to-la...,2400.0,IPO,2/1/2023 17:01,United States,Unknown,290317.0,262.018953,23%,850192.86,835.98,NaN,NaN
1,DraftKings,Boston,Consumer,140,4%,2/1/2023,https://www.cnbc.com/2023/02/01/draftkings-cut...,719.0,IPO,2/1/2023 19:08,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wheel,Austin,Healthcare,56,28%,2/1/2023,https://www.beckershospitalreview.com/teleheal...,215.0,Series C,2/2/2023 0:04,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chainalysis,New York City,Crypto,44,5%,2/1/2023,https://www.forbes.com/sites/stevenehrlich/202...,536.0,Series F,2/2/2023 3:13,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Exterro,Portland,Legal,24,3%,2/1/2023,https://www.oregonlive.com/silicon-forest/2023...,100.0,Private Equity,2/2/2023 0:05,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Inspirato,Denver,Travel,130,22%,3/16/2020,https://businessden.com/2020/03/16/inspirato-c...,79.0,Series C,3/31/2020 14:45,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1104,Help.com,Austin,Support,16,100%,3/16/2020,LinkedIn,6.0,Seed,4/16/2020 6:36,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,HopSkipDrive,Los Angeles,Transportation,8,10%,3/13/2020,https://layoffs.fyi/2020/04/02/hopskipdrive-la...,45.0,Unknown,4/3/2020 4:47,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1106,Panda Squad,SF Bay Area,Consumer,6,75%,3/13/2020,https://twitter.com/danielsinger/status/123854...,1.0,Seed,4/17/2020 0:12,United States,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
